In [17]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
from PIL import Image

In [18]:
#specifing the path to the dataset directory
#data_dir = pathlib.Path("D:/NEUB/MachineLearning/ThesisProject/x-rayImages/chest_xray/train")
data_dir = pathlib.Path("file:///D:/NEUB-Info/Thesis-Project/chest_xray/train/")

normal_dir = data_dir / "NORMAL"
pneumonia_dir = data_dir / "PNEUMONIA"

In [19]:
#counting the total images
image_count_normal = len(list(normal_dir.glob('*.jpeg')))
image_count_pneumoniya = len(list(pneumonia_dir.glob('*.jpeg')))
print(image_count_normal)
print(image_count_pneumoniya)

0
0


In [20]:
#printing the first image of normal---------
normal_images = list(normal_dir.glob('*.jpeg'))
img_normal = Image.open(normal_images[0])
img_normal.show()
#printing the first image of pneumoniya---------
pneumonia_images = list(pneumonia_dir.glob('*.jpeg'))
img_pneumoniya = Image.open(pneumonia_images[0])
img_pneumoniya.show()


IndexError: list index out of range

In [ ]:
#Load data ---------
batch_size = 32
img_height = 180
img_width = 180

val_dir = pathlib.Path("D:/NEUB/MachineLearning/ThesisProject/x-rayImages/chest_xray/val")

#loading traing data set
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
#doading validation data set
val_ds = tf.keras.utils.image_dataset_from_directory(
  val_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
cls_names = train_ds.class_names
print(cls_names)
cls_names_val = val_ds.class_names
print(cls_names_val)

In [ ]:
#Improve the training performance--------------
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#data augmentation model--------------
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomCrop(height=img_height, width=img_width),
    layers.RandomHeight(factor=0.1),
    layers.RandomWidth(factor=0.1),
    layers.RandomContrast(factor=0.1),
    #layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x)),
    #layers.Lambda(lambda x: tf.math.subtract(tf.constant(255.0), x))
  ]
)

In [ ]:
#applying data augmentation to the training data set
augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
#visualize a specific augmented image
sample_image = next(iter(train_ds))[0][6] # choose a specific image from the training dataset

fig, axs = plt.subplots(3, 3, figsize=(10, 10))
fig.suptitle("Augmented Images--")

for i, ax in enumerate(axs.flat):
    if i == 0:
        # First image is the original image
        ax.imshow(sample_image.numpy().astype("uint8"))
        ax.set_title("Original")
    else:
        # Apply the corresponding transformation to the image if the transformation index is within the range of available transformations
        if i - 1 < len(data_augmentation.layers):
            augmented_image = data_augmentation(sample_image[tf.newaxis, ...], training=True)
            ax.imshow(tf.squeeze(augmented_image).numpy().astype("uint8"))
            ax.set_title(data_augmentation.layers[i-1].name)
        else:
            # No more available transformations to apply, so leave the axis blank
            ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
num_original_images = train_ds.cardinality().numpy()*batch_size
print("Total number of images:",num_original_images)
num_transformations = 8
total_num_augmented_images = num_original_images * num_transformations
print("Total number of augmented images:",total_num_augmented_images)
num_batches = train_ds.__len__()
print("the total number of batches: ",num_batches)